# Analyzing Used Car Listings on eBay Kleinanzeigen

We will be working on a dataset of used cars from eBay Kleinanzeigen, a [classifieds](https://en.wikipedia.org/wiki/Classified_advertising) section of the German eBay website.

The dataset was originally scraped and uploaded to [Kaggle](https://www.kaggle.com/orgesleka/used-cars-database/data). The version of the dataset we are working with is a sample of 50,000 data points that was prepared by [Dataquest](https://www.dataquest.io/) including simulating a less-cleaned version of the data.

The data dictionary provided with data is as follows:

* `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
* `name` - Name of the car.
* `seller` - Whether the seller is private or a dealer.
* `offerType` - The type of listing
* `price` - The price on the ad to sell the car.
* `abtest` - Whether the listing is included in an A/B test.
* `vehicleType` - The vehicle Type.
* `yearOfRegistration` - The year in which which year the car was first registered.
* `gearbox` - The transmission type.
* `powerPS` - The power of the car in PS.
* `model` - The car model name.
* `kilometer` - How many kilometers the car has driven.
* `monthOfRegistration` - The month in which which year the car was first registered.
* `fuelType` - What type of fuel the car uses.
* `brand` - The brand of the car.
* `notRepairedDamage` - If the car has a damage which is not yet repaired.
* `dateCreated` - The date on which the eBay listing was created.
* `nrOfPictures` - The number of pictures in the ad.
* `postalCode` - The postal code for the location of the vehicle.
* `lastSeenOnline` - When the crawler saw this ad last online.

The aim of this project is to clean the data and analyze the included used car listings.

In [1]:
import pandas as pd
import numpy as np

## create a dataframe and get a basic overview of the data
autos = pd.read_csv('autos.csv', encoding='Latin-1')
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


The dataset has 20 columns, most values are stored as strings, including a few columns that represent dates/times.
5 columns have null values:
* vehicleType
* gearbox 
* model
* fuelType 
* notRepairedDamage


## Cleaning Columns

We will start by changing all column names to [snakecase](https://en.wikipedia.org/wiki/Snake_case) and changing some of the names to better describe the data, based on the data dictionary.

In [2]:
# get column names
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [3]:
col_copy=autos.columns.copy()
# create new column names 
col_copy=['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']
autos.columns=col_copy
autos.head(3)

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37


## Initial Data Exploration and Cleaning

Let's start by exploring the dataset to find areas where we can clean the data.

In [4]:
# get basic descriptive statistics of the dataset by column
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-08 10:40:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Preliminary observations:

2 columns have the same values for all or almost all rows: `seller` and `offer_type`.

Also, let's look closer at the *num_photos* column, as it looks odd:

In [5]:
autos["num_photos"].value_counts()

0    50000
Name: num_photos, dtype: int64

All values are zeros in `num_photos` column, which means we can drop it together with the other 2 columns.

In [6]:
# drop 3 columns 
autos = autos.drop(["num_photos", "seller", "offer_type"], axis=1)

Now we will work with columns `price` and `odometer` that have numeric values mixed with extra characters. We will remove the extra characters and convert them to integers.

In [7]:
# convert price column
autos["price"] = (autos["price"]
                          .str.replace("$","")
                          .str.replace(",","")
                          .astype(int)
                          )


In [8]:
# convert odometer column and rename to odometer_km to show measurement units

autos["odometer"] = (autos["odometer"]
                             .str.replace("km","")
                             .str.replace(",","")
                             .astype(int)
                             )
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)


In [9]:
autos["price"].head()


0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

In [10]:
autos['odometer_km'].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int64

## Exploring Odometer and Price

In [11]:
# frequency table
autos["odometer_km"].value_counts()

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64

We can observe that high mileage cars represent more than 80% of the dataset, and only a relatively small amount of cars have low mileage, which makes sense, since we are dealing with used cars.

Also, mileage is rounded which most likely indicates that the sellers can only choose among pre-determined set of values for that column.

In [12]:
# count of unique price points
autos['price'].value_counts().shape

(2357,)

In [13]:
autos["price"].value_counts().head(20)

0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
999      434
750      433
900      420
650      419
850      410
700      395
4500     394
300      384
2200     382
950      379
Name: price, dtype: int64

In [14]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

There are 2357 unique price points, and 1421 entries with $0 price, that is a very small % of the data, we might consider removing those rows for further analysis. 

The highest price shown is $1M dollars, which seems very odd for a used car, we should look closer at the highest values.

In [15]:
autos['price'].value_counts().sort_index(ascending=False).head(25)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
194000      1
190000      1
180000      1
175000      1
169999      1
Name: price, dtype: int64

**$350k** seems to be a price above which the prices don't make a lot of sense, they seem arbitrary, we will remove all rows with prices above that point.

In [16]:
autos['price'].value_counts().sort_index(ascending=1).head(25)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
40       6
45       4
47       1
49       4
50      49
Name: price, dtype: int64

Since ebay is an auction plaform, the bids can start from $1, so lets keep those values for now.

To conclude, we will subset out dataset based on price *(>0 and <=350000)* for further analysis



In [17]:
# subset dataset based on chosen criteria of price
autos=autos[autos['price'].between(1,350001)]

In [18]:
autos['price'].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

There are now 48565 rows of data.

## Exploring date columns

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

Right now, the `date_crawled`, `last_seen`, and `ad_created` columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively. 

The other two columns are represented as numeric values, so we can use methods like `Series.describe()` to understand the distribution without any extra data processing.

First let's look at the string columns to see how they are structured:

In [19]:
autos[['date_crawled','ad_created','last_seen']].head()

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


We will calculate the distribution of values in the `date_crawled`, `ad_created`, and `last_seen` columns (all string columns) as percentages.

In [20]:
(autos['date_crawled']
     .str[:10]
     .value_counts(normalize=True,dropna=False)
     .sort_index(ascending=True)
    )

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

Looks like the crawling happened mostly between March-April,2016, almost every day.

In [21]:
(autos["ad_created"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values(ascending=True)
        )

2016-01-13    0.000021
2016-01-29    0.000021
2016-01-07    0.000021
2016-01-14    0.000021
2015-11-10    0.000021
2016-02-09    0.000021
2016-02-22    0.000021
2016-01-16    0.000021
2015-06-11    0.000021
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-11    0.000021
2016-01-22    0.000021
2015-09-09    0.000021
2015-08-10    0.000021
2016-02-08    0.000021
2016-01-03    0.000021
2016-02-07    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-14    0.000041
2016-02-18    0.000041
2016-01-10    0.000041
2016-02-26    0.000041
2016-02-24    0.000041
2016-02-05    0.000041
2016-02-12    0.000041
2016-02-20    0.000041
                ...   
2016-03-06    0.015320
2016-03-13    0.017008
2016-03-05    0.022897
2016-03-24    0.029280
2016-03-16    0.030125
2016-03-27    0.030989
2016-03-17    0.031278
2016-03-25    0.031751
2016-03-31    0.031875
2016-03-10    0.031895
2016-03-23    0.032060
2016-03-26    0.032266
2016-03-22 

Seems like the majority of ads were created around March-April 2016, the same crawling time, distribution pretty much uniform.

In [22]:
(autos["last_seen"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values(ascending=True)
        )

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-18    0.007351
2016-03-08    0.007413
2016-03-13    0.008895
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-14    0.012602
2016-03-27    0.015649
2016-03-19    0.015834
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-26    0.016802
2016-03-23    0.018532
2016-03-25    0.019211
2016-03-24    0.019767
2016-03-21    0.020632
2016-03-20    0.020653
2016-03-28    0.020859
2016-03-22    0.021373
2016-03-29    0.022341
2016-04-01    0.022794
2016-03-31    0.023783
2016-03-12    0.023783
2016-04-04    0.024483
2016-03-30    0.024771
2016-04-02    0.024915
2016-04-03    0.025203
2016-03-17    0.028086
2016-04-05    0.124761
2016-04-07    0.131947
2016-04-06    0.221806
Name: last_seen, dtype: float64

Here we can see same timeframe, March-April 2016. Seems like most ads have a lifespan of 1-2 months.

In [23]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

The year that the car was first registered will likely indicate the age of the car. We do notice some odd values - min is 1000 and max is 9999, which means we have to clean that column.

## Cleaning Incorrect registration_year Data

Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [24]:
# get number of rows with registration year above 2016
autos.loc[autos['registration_year']>2016,'registration_year'].shape


(1879,)

In [25]:
autos.loc[autos['registration_year']>2016,'registration_year']

10       2017
55       2017
65       2017
68       2017
84       2018
113      2017
164      2018
197      2017
253      2017
348      2017
390      2018
438      2017
453      4500
454      2017
457      2017
477      2017
545      2017
548      2017
557      2017
568      2017
577      2017
582      2017
654      2017
740      2017
754      2017
758      2017
765      2017
802      2018
871      2017
889      2017
         ... 
49108    2017
49130    2017
49153    5000
49154    2017
49178    2018
49185    2019
49218    2018
49245    2017
49259    2017
49261    2017
49262    2018
49266    2017
49343    2017
49347    2017
49354    2018
49389    2017
49411    2018
49522    2017
49557    2017
49561    2017
49653    2017
49662    2017
49689    2017
49696    2017
49731    2017
49770    2018
49796    2017
49841    2017
49910    9000
49935    2017
Name: registration_year, Length: 1879, dtype: int64

There are 1879 rows with wrong data, which is less than 4% of our data - we will remove these rows.

In [26]:
# get number of rows with registration year below 1900
autos.loc[autos['registration_year']<1900,'registration_year']


10556    1800
22316    1000
24511    1111
32585    1800
49283    1001
Name: registration_year, dtype: int64

Only 5 rows - we will now also remove these rows from our dataset.

In [27]:
# subset dataset based on registration_year criteria
autos=autos[autos['registration_year'].between(1900,2016)]

In [28]:
autos['registration_year'].describe()

count    46681.000000
mean      2002.910756
std          7.185103
min       1910.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

In [29]:
(autos['registration_year']
     .value_counts(normalize=True)
     .sort_index(ascending=False)
     .head(30)
        )

2016    0.026135
2015    0.008397
2014    0.014203
2013    0.017202
2012    0.028063
2011    0.034768
2010    0.034040
2009    0.044665
2008    0.047450
2007    0.048778
2006    0.057197
2005    0.062895
2004    0.057904
2003    0.057818
2002    0.053255
2001    0.056468
2000    0.067608
1999    0.062060
1998    0.050620
1997    0.041794
1996    0.029412
1995    0.026285
1994    0.013474
1993    0.009104
1992    0.007926
1991    0.007262
1990    0.007433
1989    0.003727
1988    0.002892
1987    0.001542
Name: registration_year, dtype: float64

It appears that the majority of cars were registered in the last 22 years (1994-2016).

## Exploring Price by Brand

In [30]:
autos['brand'].describe()

count          46681
unique            40
top       volkswagen
freq            9862
Name: brand, dtype: object

In [31]:
# get top 10 brands
autos['brand'].value_counts().head(10)

volkswagen       9862
bmw              5137
opel             5022
mercedes_benz    4503
audi             4041
ford             3263
renault          2201
peugeot          1393
fiat             1197
seat              853
Name: brand, dtype: int64

40 different brands total. We see that *Volkswagen* car make is the most popular,almost double the amount of the next followers *BMW* and *OPEL*.

In [32]:
# get top 5 car brands share in the dataset
autos['brand'].value_counts(normalize=True).head(5).sum()*100

61.191919624686705

In [33]:
# get top 10 car brands share in the dataset
autos['brand'].value_counts(normalize=True).head(10).sum()*100

80.27248773590969

We can notice that the top 5 brands account for 61.2% of the whole dataset and the top 10 account for 80.3%

In [34]:
autos['brand'].value_counts(normalize=True).head(25).sum()*100

96.41181637068615

The top 25 brands account for almost entire dataset - 96,4%.
We will now analyze the average price of a car by each brand from the top 25.

In [35]:
autos['brand'].value_counts(normalize=True).head(25).index

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat', 'skoda', 'nissan', 'mazda', 'smart',
       'citroen', 'toyota', 'hyundai', 'sonstige_autos', 'volvo', 'mini',
       'mitsubishi', 'honda', 'kia', 'alfa_romeo', 'porsche'],
      dtype='object')

In [36]:
# get top 25 car labels
top25_brands=autos['brand'].value_counts().head(25).index
print(top25_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat', 'skoda', 'nissan', 'mazda', 'smart',
       'citroen', 'toyota', 'hyundai', 'sonstige_autos', 'volvo', 'mini',
       'mitsubishi', 'honda', 'kia', 'alfa_romeo', 'porsche'],
      dtype='object')


In [37]:
# Example of getting price by brand
# filter by brand
opel=autos.loc[autos['brand']=='opel']
# get price from first row
opel.iloc[0][2]

350

In [38]:
#get average price per brand example
opel['price'].mean()

2975.2419354838707

In [39]:
# Create an empty dictionary to store car brands as keys
brands_avg_price={}

In [40]:
# loop over brands and assign the avg value as value in k/v pair
for brand in top25_brands:
    select_rows=autos.loc[autos['brand']==brand]
    avg_price=select_rows['price'].mean()
    brands_avg_price[brand]=int(avg_price)
    
brands_avg_price


{'alfa_romeo': 4087,
 'audi': 9336,
 'bmw': 8332,
 'citroen': 3779,
 'fiat': 2813,
 'ford': 3749,
 'honda': 4107,
 'hyundai': 5365,
 'kia': 5982,
 'mazda': 4112,
 'mercedes_benz': 8628,
 'mini': 10613,
 'mitsubishi': 3394,
 'nissan': 4743,
 'opel': 2975,
 'peugeot': 3094,
 'porsche': 45643,
 'renault': 2474,
 'seat': 4397,
 'skoda': 6368,
 'smart': 3580,
 'sonstige_autos': 12338,
 'toyota': 5167,
 'volkswagen': 5402,
 'volvo': 4946}

Since dictionaries store data randomly, in the next step we will create a list of lists with price and brand name in each list, and then sort by descending order.

In [41]:
# create empty list
avg_price_list=[]
#fill the list with lists [price,brand]
for b in brands_avg_price:
    price=[brands_avg_price[b],b]
    avg_price_list.append(price)

sorted(avg_price_list,reverse=True)

[[45643, 'porsche'],
 [12338, 'sonstige_autos'],
 [10613, 'mini'],
 [9336, 'audi'],
 [8628, 'mercedes_benz'],
 [8332, 'bmw'],
 [6368, 'skoda'],
 [5982, 'kia'],
 [5402, 'volkswagen'],
 [5365, 'hyundai'],
 [5167, 'toyota'],
 [4946, 'volvo'],
 [4743, 'nissan'],
 [4397, 'seat'],
 [4112, 'mazda'],
 [4107, 'honda'],
 [4087, 'alfa_romeo'],
 [3779, 'citroen'],
 [3749, 'ford'],
 [3580, 'smart'],
 [3394, 'mitsubishi'],
 [3094, 'peugeot'],
 [2975, 'opel'],
 [2813, 'fiat'],
 [2474, 'renault']]

As we can see, the highest average price belongs to Porshe, which is not a surprise, as it is positioned as a luxury car brand. Followed by a substantial gap are Sonstige Autos, Mini, Audi and Mercedes Benz. Volkswagen's average price is $5,4k - this might be one of the factors explaining the popularity, since it is also a trusted well-known brand.

The least expensive are Opel, Fiat and Renault, falling below $3k on average.

## Exploring Mileage

In [42]:
# Create an empty dictionary to store car brands as keys
brands_avg_mil={}

# loop over brands and assign the avg mileage as value in k/v pair
for brand in top25_brands:
    select_rows=autos.loc[autos['brand']==brand]
    avg_mil=select_rows['odometer_km'].mean()
    brands_avg_mil[brand]=int(avg_mil)
    
brands_avg_mil


{'alfa_romeo': 131338,
 'audi': 129157,
 'bmw': 132572,
 'citroen': 119694,
 'fiat': 117121,
 'ford': 124266,
 'honda': 122172,
 'hyundai': 106442,
 'kia': 112530,
 'mazda': 124464,
 'mercedes_benz': 130788,
 'mini': 88105,
 'mitsubishi': 126575,
 'nissan': 118330,
 'opel': 129310,
 'peugeot': 127153,
 'porsche': 96853,
 'renault': 128071,
 'seat': 121131,
 'skoda': 110848,
 'smart': 99326,
 'sonstige_autos': 89956,
 'toyota': 115944,
 'volkswagen': 128707,
 'volvo': 138067}

In [43]:
# create empty list
avg_mil_list=[]
#fill the list with lists [mileage,brand]
for m in brands_avg_mil:
    mil=[brands_avg_mil[m],m]
    avg_mil_list.append(mil)

sorted(avg_mil_list,reverse=True)

[[138067, 'volvo'],
 [132572, 'bmw'],
 [131338, 'alfa_romeo'],
 [130788, 'mercedes_benz'],
 [129310, 'opel'],
 [129157, 'audi'],
 [128707, 'volkswagen'],
 [128071, 'renault'],
 [127153, 'peugeot'],
 [126575, 'mitsubishi'],
 [124464, 'mazda'],
 [124266, 'ford'],
 [122172, 'honda'],
 [121131, 'seat'],
 [119694, 'citroen'],
 [118330, 'nissan'],
 [117121, 'fiat'],
 [115944, 'toyota'],
 [112530, 'kia'],
 [110848, 'skoda'],
 [106442, 'hyundai'],
 [99326, 'smart'],
 [96853, 'porsche'],
 [89956, 'sonstige_autos'],
 [88105, 'mini']]

The highest average mileage is observed in Volvo cars. Volkswagen ranks #7 on the list, with an average of 128,071 miles. The mileage for the most expensive on average car - Porshe is ranked #23.

Now let's create a new dataframe that captures the top 25 cars' average price and mileage:

In [73]:
# convert dictionaries to Series object
price_series=pd.Series(brands_avg_price).sort_values(ascending=False)
mileage_series=pd.Series(brands_avg_mil).sort_values(ascending=False)


In [74]:
#convert to Dataframe object
df1=pd.DataFrame(price_series,columns=['avg_price'])
df2=pd.DataFrame(mileage_series,columns=['avg_mileage'])
# combine 2 DFs in one
df1['avg_mileage']=df2

In [75]:
df1

,avg_price,avg_mileage
porsche,45643,96853
sonstige_autos,12338,89956
mini,10613,88105
audi,9336,129157
mercedes_benz,8628,130788
bmw,8332,132572
skoda,6368,110848
kia,5982,112530
volkswagen,5402,128707
hyundai,5365,106442


The top 3 brands by average price have the lowest mileage, the rest seem to fall within a similar range, and the mileage jump up by 30-35k on average.